# How to match? 

## 0. Import libraries 

In [22]:
import pandas as pd
import warnings
import numpy as np
from lightautoml.addons.hypex import Matcher

warnings.filterwarnings('ignore')
%config Completer.use_jedi = False

## 1. Create or upload your dataset  
In this case we will create random dataset with known effect size  
If you have your own dataset, go to the part 2 


In [23]:
# Simulating dataset with known effect size
num_users = 10000
num_months = 12

signup_months = np.random.choice(np.arange(1, num_months), num_users) * np.random.randint(0,2, size=num_users) # signup_months == 0 means customer did not sign up
df = pd.DataFrame({
    'user_id': np.repeat(np.arange(num_users), num_months),
    'signup_month': np.repeat(signup_months, num_months), # signup month == 0 means customer did not sign up
    'month': np.tile(np.arange(1, num_months+1), num_users), # months are from 1 to 12
    'spend': np.random.poisson(500, num_users*num_months) #np.random.beta(a=2, b=5, size=num_users * num_months)*1000 # centered at 500
})
# A customer is in the treatment group if and only if they signed up
df["treat"] = df["signup_month"]>0
# Simulating an effect of month (monotonically decreasing--customers buy less later in the year)
df["spend"] = df["spend"] - df["month"]*10
# Simulating a simple treatment effect of 100
after_signup = (df["signup_month"] < df["month"]) & (df["treat"])
df.loc[after_signup,"spend"] = df[after_signup]["spend"] + 100
df.head()

,user_id,signup_month,month,spend,treat
0,0,11,1,494,True
1,0,11,2,451,True
2,0,11,3,459,True
3,0,11,4,480,True
4,0,11,5,444,True


In [24]:
# Setting the signup month (for ease of analysis)
i = 3
df_i_signupmonth = (
    df[df.signup_month.isin([0, i])]
    .groupby(["user_id", "signup_month", "treat"])
    .apply(
        lambda x: pd.Series(
            {
                "pre_spends": x.loc[x.month < i, "spend"].mean(),
                "post_spends": x.loc[x.month > i, "spend"].mean(),
            }
        )
    )
    .reset_index()
)
df_i_signupmonth

,user_id,signup_month,treat,pre_spends,post_spends
0,1,0,False,484.0,424.111111
1,2,0,False,488.5,425.222222
2,3,0,False,488.5,412.444444
3,4,0,False,493.5,422.888889
4,8,0,False,476.0,410.111111
...,...,...,...,...,...
5476,9990,0,False,453.0,423.666667
5477,9991,0,False,477.0,424.111111
5478,9995,0,False,517.0,437.111111
5479,9997,3,True,469.5,517.222222


In [25]:
# Additional category features
gender = np.random.choice(a=[0,1], size=df_i_signupmonth.user_id.nunique())
age = np.random.choice(a=range(18, 70), size=df_i_signupmonth.user_id.nunique())
industry = np.random.choice(a=range(1, 3), size=df_i_signupmonth.user_id.nunique())
df_i_signupmonth['age'] = age
df_i_signupmonth['is_male'] =  gender
df_i_signupmonth['industry'] =  industry
df_i_signupmonth['industry'] = df_i_signupmonth['industry'].astype('str')
df_i_signupmonth['treat'] = df_i_signupmonth['treat'].astype(int)
df_i_signupmonth.head()

,user_id,signup_month,treat,pre_spends,post_spends,age,is_male,industry
0,1,0,0,484.0,424.111111,41,1,1
1,2,0,0,488.5,425.222222,19,0,1
2,3,0,0,488.5,412.444444,23,0,2
3,4,0,0,493.5,422.888889,60,1,1
4,8,0,0,476.0,410.111111,26,0,2


In [26]:
df_i_signupmonth.columns

Index(['user_id', 'signup_month', 'treat', 'pre_spends', 'post_spends', 'age',
       'is_male', 'industry'],
      dtype='object')

## 2. Matching  
### 2.0 Init params
info_col used to define informative attributes that should not be part of matching, such as user_id  
But to explicitly store this column in the table, so that you can compare directly after computation

In [27]:
info_col = ['user_id', 'signup_month']

outcome = 'post_spends'
treatment = 'treat'

### 2.1 Simple matching
This is the easiest way to initialize and calculate metrics on a Matching task  
Use it when you are clear about each attribute or if you don't have any additional task conditions (Strict equality for certain features) 

In [28]:
# Standard model with base parameters
model = Matcher(input_data=df_i_signupmonth, outcome=outcome, treatment=treatment, info_col=info_col)
results, quality_results = model.estimate()

  0%|          | 0/5481 [00:00<?, ?it/s]

### 2.2 Matching with a fixed variable  
Used when you have categorical feature(s) that you want to compare by strict equality  
group_col is used for strict comparison of categorical features.  
In our case there is only one attribute  
If there are several such attributes, you should make one of them and use it

In [29]:
# group_col is used for strict comparison of categorical features.  
# In our case there is only one attribute  
# If there are several such attributes, you should make one of them and use it

group_col = "industry"

In [30]:
model = Matcher(input_data=df_i_signupmonth, outcome=outcome, treatment=treatment,
                info_col=info_col, group_col=group_col)
results, quality_results = model.estimate()

  0%|          | 0/4 [00:00<?, ?it/s]

### 2.3 Matching but you don't know which features are better to use  
This is the easiest way to initialize and calculate metrics on a Matching task  
Use this method when you want to select the most important features and make matching based on them. 

In [31]:
model = Matcher(input_data=df_i_signupmonth, outcome=outcome, treatment=treatment,
                info_col=info_col, group_col=group_col)

In [32]:
features_importance = model.lama_feature_select()
features_importance

,Feature,Importance
0,pre_spends,27605.289307
2,industry,27605.289307
1,age,24612.019775


In [33]:
features = features_importance['Feature'].to_list()

In [34]:
# You can use both variant that you like. So you can just list the features to be matched

#results, quality_results = model.estimate(features=features_importance[:3])
results, quality_results = model.estimate(features=features[:3])

  0%|          | 0/4 [00:00<?, ?it/s]

In [35]:
model.matcher.df_matched

,index,pre_spends,age,is_male,industry,pre_spends_matched,age_matched,is_male_matched,index_matched,industry_matched,post_spends,post_spends_matched,post_spends_matched_bias,treat,treat_matched
0,5018,497.0,38,1,1,497.0,38.0,0.5,"[1825, 2085]",1,522.222222,421.888889,100.333333,1,0
1,5019,465.0,29,1,1,465.0,30.0,0.0,[2186],1,500.444444,424.222222,76.207975,1,0
2,5020,492.5,44,0,1,492.0,44.0,1.0,[1764],1,525.555556,413.333333,112.221298,1,0
3,5021,498.0,50,0,1,498.0,50.0,0.0,[1741],1,525.444444,407.333333,118.111111,1,0
4,5022,493.5,69,0,1,493.0,69.0,0.0,[1955],1,507.555556,427.000000,80.554631,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5013,5013,507.0,31,1,2,509.5,33.0,1.0,[439],2,425.888889,523.000000,97.200151,0,1
5014,5014,494.5,37,1,2,495.5,37.0,0.0,[401],2,422.000000,530.111111,108.141195,0,1
5015,5015,465.0,52,1,2,463.0,55.0,1.0,[226],2,425.111111,517.555556,92.405020,0,1
5016,5016,488.0,51,1,2,488.5,51.0,0.0,[378],2,417.555556,520.333333,102.792820,0,1


## 3. Results  
### 3.1 ATE, ATT, ATC

In [36]:
# model.matcher.results
results

,effect_size,std_err,p-val,ci_lower,ci_upper
ATE,100.790527,0.592346,0.0,99.629529,101.951524
ATC,100.826633,0.603504,0.0,99.643764,102.009501
ATT,100.399209,0.710848,0.0,99.005946,101.792472


### 3.2 SMD, PSI, KS-test, repeats

In [37]:
# matching quality result - SMD
model.quality_result

{'psi':         column_treated  anomaly_score_treated check_result_treated  \
 0          age_treated                   0.00                   OK   
 1     industry_treated                   0.00                   OK   
 2  post_spends_treated                  16.11                  NOK   
 3   pre_spends_treated                   0.00                   OK   
 
         column_untreated  anomaly_score_untreated check_result_untreated  
 0          age_untreated                     0.01                     OK  
 1     industry_untreated                     0.00                     OK  
 2  post_spends_untreated                     8.23                    NOK  
 3   pre_spends_untreated                     0.01                     OK  ,
 'ks_test':             match_control_to_treat  match_treat_to_control
 age                       1.000000                0.530347
 is_male                   0.218892                0.000008
 pre_spends                1.000000                0.130669,
 's

In [38]:
# matching quality result - PSI
model.quality_result['psi']

,column_treated,anomaly_score_treated,check_result_treated,column_untreated,anomaly_score_untreated,check_result_untreated
0,age_treated,0.00,OK,age_untreated,0.01,OK
1,industry_treated,0.00,OK,industry_untreated,0.00,OK
2,post_spends_treated,16.11,NOK,post_spends_untreated,8.23,NOK
3,pre_spends_treated,0.00,OK,pre_spends_untreated,0.01,OK


In [39]:
# matching quality result - KS-test

model.quality_result['ks_test']

,match_control_to_treat,match_treat_to_control
age,1.000000,0.530347
is_male,0.218892,0.000008
pre_spends,1.000000,0.130669


In [40]:
# matching quality result - repeats
model.quality_result['repeats']

{'match_control_to_treat': 0.93, 'match_treat_to_control': 0.09}

### 3.3 Validation
validate result with placebo treatment  
our new effect size is close to zero (it should be)

In [41]:
 
model.validate_result()
 

  0%|          | 0/10 [00:00<?, ?it/s]

{'post_spends': [100.33542878123436, 4.763074247238208e-06]}

In [42]:
model.matcher.df_matched

,index,pre_spends,age,is_male,industry,random_feature,pre_spends_matched,age_matched,is_male_matched,random_feature_matched,index_matched,industry_matched,post_spends,post_spends_matched,post_spends_matched_bias,treat,treat_matched
0,5018,497.0,38,1,1,-1.752593,499.5,37.0,0.0,-1.716658,[1935],1,522.222222,423.333333,98.913238,1,0
1,5019,465.0,29,1,1,0.235149,464.0,30.0,1.0,0.341418,[581],1,500.444444,429.888889,70.463467,1,0
2,5020,492.5,44,0,1,-0.802581,491.5,44.0,1.0,-0.749453,[588],1,525.555556,430.111111,95.395304,1,0
3,5021,498.0,50,0,1,1.081684,496.5,52.0,1.0,1.182453,[1438],1,525.444444,418.444444,106.894707,1,0
4,5022,493.5,69,0,1,-0.608776,495.0,69.0,1.0,-0.650500,[2205],1,507.555556,422.666667,84.938396,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5013,5013,507.0,31,1,2,0.620057,509.5,33.0,1.0,1.038798,[439],2,425.888889,523.000000,96.837168,0,1
5014,5014,494.5,37,1,2,1.161467,495.5,37.0,0.0,1.194265,[401],2,422.000000,530.111111,108.119833,0,1
5015,5015,465.0,52,1,2,1.381204,460.5,51.0,1.0,1.589511,[441],2,425.111111,514.444444,88.976178,0,1
5016,5016,488.0,51,1,2,0.875593,487.5,50.0,0.0,1.134119,[233],2,417.555556,525.555556,107.749673,0,1
